In [1]:
import urllib

In [7]:
#urllib.parse.urlencode("http://google.com/q=this is a cat")

In [80]:
import pandas as pd
import numpy as np
import json
from datetime import datetime 


def parse_json_file_to_csv(fname): 
    keyz = [ 'prism:url', 
             "dc:identifier", "dc:title", "prism:publicationName",
             "prism:coverDate", "citedby-count", 
            ]
    affilz = "affiliation" ## sub = affilname, affiliation-city, affiliation-country
    first_affilz_country = "country_1"
    first_affilz_org = "institution_1"
    
    has_fundus_in_title = "fundus_in_title"
    pub_year = "pub_year"
    
    abhref = 'abstract'
    
    def parse_elsevier_json_item(item):    
        
        def get_pub_year(pub_date):
            return datetime.strptime(pub_date, "%Y-%m-%d").year
        
        def has_fundus(titl):
            return str( int( 'fundus' in titl.lower() ) )
        
        O_ = []
        ## 1. everything top level 
        for k in keyz:
            O_.append( item[k] ) 
        
        ## 2. parse affiliation details and join org,city,country
        k_affz = ['affilname', 'affiliation-city', 'affiliation-country']
        A_ = []
        I_ = '-'
        C_ = '-'
        ia = 0 
        affitem = item.get("affiliation", None)
        if affitem:
            for aff in affitem:
                if aff:
                    a_ = []
                    for k in k_affz:
                        x = aff.get(k, "-") 
                        a_.append( x if x is not None else "-")
#                     print( a_ )
                    A_.append(", ".join(a_) )         
                    if ia == 0:
                        C_ = a_[-1]  
                        I_ = a_[0]
                        ia = 100 
            O_.append( "++".join(A_) )
        else:
            O_.append( "-" )
        O_.append( I_ )
        O_.append( C_ )
        
        ## 3. preprocs
        O_.append( has_fundus( item.get("dc:title", "") ) )
        O_.append( get_pub_year(item.get("prism:coverDate", "")))
        
        ## 4. parse abstract url 
        abstract = "link" # @ref = "self", "@href" 
        for abl in item[abstract]:
            if abl['@ref'] == 'self':
                O_.append( abl['@href'] ) 
        
        return O_ 
    
        
    def fetch_abstract(ahref):
        pass 
    
    to_csv = []
    with open( f'{fname}.json', 'r') as fd:
        pagez = json.load( fd)
#         print( type(pagez), len(pagez) )
        for page in pagez:
            for item in page:
                to_csv.append( parse_elsevier_json_item(item) )
            
    
    def dump_csv_row(rec):
#         print(type(rec), len(rec), rec)
        fd.write( "\t".join( rec) )
        fd.write("\n")
        
    with open( f"{fname}.csv", 'w') as fd:
        headz = keyz + [affilz, first_affilz_org, first_affilz_country, 
                        has_fundus_in_title, pub_year, 
                        abhref] 
        dump_csv_row(headz)
        for rec in to_csv:
            dump_csv_row(rec)
            
        

In [81]:
datz = ['0ea_4__fuse_hc_dl', 
        '0ea_5__fundus_hc_dl', 
        '0ea_642__medical_img_preproc', 
        '0ea_3926__fundus'
       ]

for fname in datz:
    parse_json_file_to_csv(fname)
    print("******** FINISHED - ", fname )

TypeError: sequence item 9: expected str instance, int found

In [69]:
abid="https://api.elsevier.com/content/abstract/scopus_id/85098927137"

In [77]:
from datetime import datetime
datetime.strptime("2021-09-01", "%Y-%m-%d").year

2021

In [86]:
str(int('fundus' in "The quick brown".lower() ) )

'0'